In [1]:
import sys
sys.path.insert(0, '../code/')
from dismod_db import DismodDB
from two_level import TwoLevel
from plot_two_level import PlotTwoLevel
import numpy as np
import pandas as pd
from dismod_output import DismodOutput

#### Build database

In [2]:
data = pd.read_csv('./data/diabetes_sinc_complete_loc.csv')

In [3]:
data.head()

,Unnamed: 0,location_id,X,sex,age_start,age_end,year_start,year_end,mean,standard_error,measure,ow,bmi,fpg,location_name,region_name
0,1,8,855,Female,30,44.0,2001,2002,0.000025,0.0001,Sincidence,0.271136,23.525415,5.100081,Taiwan,East Asia
1,2,8,859,Male,45,59.0,2007,2008,0.000005,0.0001,Sincidence,0.538278,24.704975,5.562921,Taiwan,East Asia
2,3,8,879,Female,30,44.0,2007,2008,0.000017,0.0001,Sincidence,0.306358,24.154000,5.032846,Taiwan,East Asia
3,4,8,883,Female,0,14.0,2003,2004,0.000058,0.0001,Sincidence,0.181352,23.763450,0.000000,Taiwan,East Asia
4,5,8,857,Female,0,14.0,2005,2006,0.000055,0.0001,Sincidence,0.184948,23.969006,0.000000,Taiwan,East Asia


In [4]:
integrand = ['Sincidence']
locations = ['Australia', 'New Zealand']
rates = ['iota']
meas_noise_density = {'Sincidence': {'density': 'gaussian'}}
rates_parent_priors = [
    # rate priors for iota
    ({'density':'uniform','upper':1.0,'lower':1e-16,'mean': .0001},\
    {'density': 'gaussian', 'mean':0.0, 'std': .001},\
    {'density': 'gaussian', 'mean':0.0, 'std': .001})]
rates_child_priors = [({'density':'gaussian','mean': 0.0, 'std': .1}, None, None)]
covariates = [({'name':'bmi','type': 'rate_value','effected': 'iota'},None),
             ({'name': 'ow', 'type': 'rate_value', 'effected': 'iota'}, None),
             ({'name': 'fpg', 'type': 'rate_value', 'effected': 'iota'}, [25, 100])]
cov_priors = [{'density':'uniform','mean': 0.0, 'lower': -0.1, 'upper': 0.1}, 
              {'density':'uniform','mean': 0.0, 'lower': -10.0, 'upper': 10.0},
             {'density':'uniform','mean': 0.0, 'lower': -1.0, 'upper': 1.0}]
#---------change path------------
path_to_db = './data/incidence_two_level_cov_test.db'

In [20]:
db = DismodDB(data, locations, integrand,rates, rates_parent_priors, 
              rates_child_priors, meas_noise_density, path_to_db, 
              covariates=covariates, cov_priors=cov_priors)

#### Train model

In [21]:
model = TwoLevel(db)

In [22]:
model.fit_both(tol=1e-8, max_iter=20, fit_fixed=False)

/home/prefix/dismod_at.release/bin/dismod_at ./data/incidence_two_level_cov_test.db init
./data/incidence_two_level_cov_test.db fit both
./data/incidence_two_level_cov_test.db predict fit_var


In [24]:
output = DismodOutput(path_to_db)
output.get_covariate_multiplier_values(['bmi', 'ow', 'fpg'])

,age,covariate_name,fit_var_value
3,0.0,bmi,-0.087295
4,100.0,bmi,-0.087295
5,0.0,ow,0.400607
6,100.0,ow,0.400607
7,0.0,fpg,0.000000
8,25.0,fpg,-0.133002
9,100.0,fpg,-0.133002
